# HMM Exercise

#### In this exercise, we will use the ``hmmlearn`` library to practice what we have learned about Hidden Markov Models (HMMs), both on a toy problem and to build a POS tagger from scratch. First make sure that ``hmmlearn`` is installed:

In [1]:
pip install hmmlearn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from hmmlearn.hmm import MultinomialHMM
from sklearn.metrics import  accuracy_score

from collections import Counter
import nltk
nltk.download('brown')
nltk.download('universal_tagset')
import warnings
warnings.simplefilter(action='ignore')
warnings.filterwarnings('ignore', category=ImportWarning)


[nltk_data] Downloading package brown to /home/gal/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/gal/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


# Part 1: Seasonal weather

**In this section, we will consider the following toy problem:**

**A year has four seasons - winter, spring, summer, and fall. Every day of the year we go outside, look at the sky to see if it is sunny or cloudy, and try to guess what season it is.**

## Questions:*
### 1. If we model this with an HMM, what are the hidden states? What are the observed variables?


In [2]:
hidden_states = {'winter':0, 'spring':1, 'summer':2, 'fall':3} #n_components
 
observed_variables = {'rainy':0, 'sunny':1} #n_features

### 2. Using ``hmmlearn.hmm.MultinomialHMM``, create an HMM model for this problem. Note: you may refer to [the hmmlearn API documentation](https://hmmlearn.readthedocs.io/en/latest/api.html).


####  * Instantiate the HMM model with ``weather_model = MultinomialHMM(n_components=...)``, where ``n_components`` is set to the number of hidden states.


In [3]:
weather_model = MultinomialHMM(n_components=4)

####  * Set ``weather_model.startprob_`` so that the model starts in winter (probability 1 for winter and 0 for other seasons). Hint: ``weather_model.startprob_`` should be a list of numbers.


In [4]:
weather_model.startprob_ = [1, 0, 0, 0]

####  * Set ``weather_model.transmat_`` so that on every day, there is a 0.99 probability of the season remaining the same as the previous day and a 0.01 probability of it being the next season. Hint: ``model.transmat_`` should be a NumPy array of shape ``(n_components, n_components)``


In [5]:
weather_model.transmat_ = np.array([[0.99, 0.01, 0, 0],
                                    [0, 0.99, 0.01, 0],
                                    [0, 0, 0.99, 0.01],
                                    [0.01, 0, 0, 0.99]])

####  * Set ``weather_model.emissionprob_`` so that 90% of summer days are sunny, 90% of winter days are rainy, and 50% of spring and fall days are sunny or rainy. Hint: ``model.emissionprob_`` should be a NumPy array of shape ``(n_components, n_features)``.


In [6]:
weather_model.emissionprob_ = np.array([[0.1, 0.9],
                                        [0.5, 0.5],
                                        [0.9, 0.1],
                                        [0.5, 0.5]])

### 3. Sample 100 days from the model by using  ``weather_model.sample(100)``.


In [37]:
sample_matrix, state_sequence = weather_model.sample(100)

 #### How many sunny days are observed?

In [38]:
len(sample_matrix[sample_matrix == 0])

44

 #### How many times did the season change?

In [79]:
#first way - simple
count = 0
for i in range(1,len(state_sequence)):
    if state_sequence[i-1] != state_sequence[i]:
        count+=1
print(count)

5


In [97]:
#second way from the box
from more_itertools import unique_justseen
"""List unique elements, preserving order. Remember only the element just seen."
    # unique_justseen('AAAABBBCCDAABBB') --> A B C D A B
    # unique_justseen('ABBCcAD', str.lower) --> A B C A D"""

len(list(unique_justseen(state_sequence)))-1

5

In [81]:
state_sequence

[0 0 0 0 0 0 0 0 0 1 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


We see the changes from 0 to 1 and from 1 to 2, i.e. It was a season change from winter to spring and from spring to summer

### 4. If 50 sunny days and then 50 rainy days are observed, what is the most likely sequence of seasons for those days under this model? Use ``weather_model.decode(...)`` to determine this. Hint: The input to this function should be an array of shape`` (100, 1)`` .

In [12]:
arr_days = np.concatenate((np.zeros(50, dtype=int), np.ones(50, dtype=int))).reshape(-1,1)
arr_days.shape

(100, 1)

In [13]:
logprob, seq = weather_model.decode(arr_days)
logprob, seq

(-33.2843121229581,
 array([0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

We see, that the majority is 0 and 2, i.e. winter and summer

In [14]:
uni_var, val_count = np.unique(seq, return_counts = True)

dict(zip(uni_var,val_count))

{0: 51, 1: 1, 2: 47, 3: 1}

# Part 2: Building a POS tagger

I**n this section, we will build a simple Part-of-Speech (POS) tagger for English texts based on labelled data from the [Brown Corpus](https://en.wikipedia.org/wiki/Brown_Corpus). First make sure that the necessary corpora are downloaded:**

**Now we will build a HMM-based POS tagger from scratch, step-by-step:**
## Questions:

### 5. Get POS-tagged sentences from the Brown corpus using 

``nltk.corpus.brown.tagged_sents(tagset='universal')``.

In [15]:
data = nltk.corpus.brown.tagged_sents(tagset='universal')

#### Use ``sklearn.model_selection.train_test_split`` to split them into 80% training data and 20% testing data.


In [16]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [17]:
len(train), len(test)

(45872, 11468)

In [18]:
len(train)/len(data), len(test)/len(data)

(0.8, 0.2)

### 6. Define ``pos_tags`` to be an array of all unique POS tags found in the training data. 


In [19]:
pos_tags = (set([tag for item in train for word, tag in item]))
pos_tags

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

#### How many unique POS tags were found?

In [20]:
len(pos_tags)

12

### 7. Using collections.Counter, find the 5000 most common word tokens in the training data, and save their unique values to an array ``vocab``. Make all words lowercase before counting, and in addition, add the token '[UNK]' as the first element of ``vocab`` (representing words which are "out of vocabulary"). Hint: the first five elements of ``vocab`` should be \["\[UNK\]", "the", ",", ".", "of", ...\]


In [21]:
vocab = Counter([word.lower() for item in train for word, tag in item]).most_common(5000)
vocab = [word[0] for word in vocab]
vocab.insert(0,"[UNK]")
vocab[:5]

['[UNK]', 'the', ',', '.', 'of']

### 8. Using ``hmmlearn.hmm.MultinomialHMM``, create an HMM model ``pos_model`` for POS tagging. It will have POS tags as hidden states and English word tokens as outputs. 


In [22]:
pos_model = MultinomialHMM(n_components=len(pos_tags))

###  * Set ``pos_model.startprob_`` by using the fraction of sentences in the training data starting with a word with each given POS tag (i.e. what fraction of sentences start with a noun? What fraction of them start with a verb? Etc.) Hint: This should be a list of length ``len(pos_tags)``, of probabilities which sum to one.


**source:** https://www.nltk.org/book/ch05.html#ref-dict-to-list

List of non-unique tags in train set

In [23]:
tags = [tag for item in train for word, tag in item]
len(tags)

929265

In [24]:
tags[:10]

['NOUN', '.', 'DET', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET']

Dictionary tag: count sorted by key

In [25]:
tags_count={ k:0 for k in sorted(pos_tags) }
for tup in train:
    tags_count[tup[0][1]] +=1
tags_count

{'.': 4091,
 'ADJ': 1558,
 'ADP': 5678,
 'ADV': 4136,
 'CONJ': 2222,
 'DET': 9771,
 'NOUN': 6465,
 'NUM': 769,
 'PRON': 7351,
 'PRT': 1700,
 'VERB': 2113,
 'X': 18}

Definding the dictionary with tag probabilities

In [26]:
f = ({k:v/len(train) for k,v in sorted(tags_count.items())})
f

{'.': 0.08918294384373911,
 'ADJ': 0.03396407394489013,
 'ADP': 0.12377921171956749,
 'ADV': 0.09016393442622951,
 'CONJ': 0.048439134984304154,
 'DET': 0.21300575514475062,
 'NOUN': 0.14093564701778863,
 'NUM': 0.016764039065224973,
 'PRON': 0.16025026159748867,
 'PRT': 0.037059644227415416,
 'VERB': 0.04606295779560516,
 'X': 0.00039239623299616326}

Writing into pos_model.startprob_ values of the dictionary with tag probabilities

In [27]:
pos_model.startprob_ = list(f.values())
pos_model.startprob_

[0.08918294384373911,
 0.03396407394489013,
 0.12377921171956749,
 0.09016393442622951,
 0.048439134984304154,
 0.21300575514475062,
 0.14093564701778863,
 0.016764039065224973,
 0.16025026159748867,
 0.037059644227415416,
 0.04606295779560516,
 0.00039239623299616326]

Check for sum == 1

In [28]:
np.sum(pos_model.startprob_)

1.0

 ### * Set ``pos_model.transmat_`` to be the probabilities of transitioning from one POS tag to another, estimated from the training data samples. Hint: This is a matrix of shape ``(len(pos_tags), len(pos_tags))`` whose rows each sum to one.


**source:** https://stackoverflow.com/questions/47297585/building-a-transition-matrix-using-words-in-python-numpy

Create the dataframe - transition matrix for tags

In [29]:
ct = pd.crosstab(pd.Series(tags[:-1]),pd.Series(tags[1:]),normalize=0)
ct

col_0,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
row_0,,,,,,,,,,,,
.,0.141134,0.041849,0.112263,0.077801,0.087984,0.148648,0.136508,0.018476,0.107053,0.032556,0.094405,0.001322
ADJ,0.099449,0.056864,0.087948,0.009754,0.038283,0.006034,0.653348,0.007065,0.004033,0.019388,0.017371,0.000463
ADP,0.009819,0.082389,0.019923,0.015578,0.001836,0.454913,0.259375,0.030242,0.069570,0.014336,0.041518,0.000500
ADV,0.170299,0.136608,0.142368,0.097114,0.017635,0.073608,0.032846,0.013321,0.047456,0.028531,0.240126,0.000089
CONJ,0.020965,0.113317,0.072889,0.091993,0.000229,0.150578,0.242571,0.019300,0.067207,0.025047,0.195448,0.000457
DET,0.012607,0.239929,0.009202,0.017582,0.000621,0.005961,0.626099,0.009850,0.009877,0.002100,0.064804,0.001369
NOUN,0.284059,0.012848,0.245452,0.026670,0.059999,0.016209,0.150256,0.008055,0.020142,0.018062,0.157885,0.000362
NUM,0.270584,0.058681,0.132157,0.020229,0.039789,0.014879,0.381760,0.021650,0.008944,0.005684,0.045390,0.000251
PRON,0.103173,0.009442,0.055102,0.053731,0.011371,0.017284,0.008807,0.001142,0.008629,0.024162,0.707132,0.000025


Writing down the values into pos_model.transmat_

In [30]:
pos_model.transmat_ = ct.values

Check for `sum of row` = 1

In [31]:
pos_model.transmat_[0].sum()

1.0

 Check the shape is the same as in a question's hint

In [32]:
(len(pos_tags), len(pos_tags)) == (pos_model.transmat_).shape

True

###  * Set ``pos_model.emissionprob_`` to be the probabilities of outputting each word in ``vocab`` given a POS tag. Estimate this from the training data samples, making sure to make all words lowercase and to replace words that are not in ``vocab`` with "\[UNK\]". Hint: This is a matrix of shape ``(len(pos_tags), len(vocab))`` whose rows each sum to one.


Prepare the zero matrix 

In [33]:
emission_matrix = defaultdict(int,{k:np.zeros(len(pos_tags)) for k in vocab})

Creating the dictionary with indx for tags

In [34]:
tag_indx = defaultdict(int,{k:i for i,k in enumerate(sorted(pos_tags))})

Creating the dictionary with indx for words in vocab

In [35]:
words_indx = defaultdict(int,{k:i for i,k in enumerate(vocab)})

Set pos_model.emissionprob_ to be the probabilities of outputting each word in vocab given a POS tag

In [36]:
for tup in train:
    for wrd, tg in tup:
        tag = tag_indx[tg]
        if wrd.lower() in emission_matrix.keys():
            emission_matrix[wrd.lower()][tag] +=1
        else:
            emission_matrix['[UNK]'][tag] +=1

for key in emission_matrix.keys():
    summ = np.sum(emission_matrix[key])
    if summ == 0:
        print(key, emission_matrix[key])
    else:
        emission_matrix[key] = emission_matrix[key]/summ

Check the sum == 1

In [37]:
np.array(list(emission_matrix.values()))[0].sum()

1.0

In [38]:
pos_model.emissionprob_ = np.array(list(emission_matrix.values())).T

Check the shape of emissionprob matrix with conditions in question

In [39]:
pos_model.emissionprob_.shape

(12, 5001)

In [40]:
pos_model.emissionprob_.shape == (len(pos_tags), len(vocab))

True

### 9. Make a function ``get_pos(sentence)`` that returns the most likely POS tags for the words in the string ``sentence``, calculated using pos_model.decode(...). Apply this to a few (lowercase, no punctuation) sentences including: "this is a test", "tel aviv is in israel", "i know how to write code". 


In [41]:
def get_pos(sentence):
    sentence_list = sentence.split() 
    x = np.array([words_indx[word] for word in sentence_list]).reshape(-1, 1)
    _, tag_decode = pos_model.decode(x)
    pos_list =[]
    for i in tag_decode:
        pos_list.append([u for u in tag_indx if tag_indx[u] == i])
    return pos_list

In [42]:
get_pos("this is a test")

[['DET'], ['VERB'], ['DET'], ['NOUN']]

In [43]:
get_pos("tel aviv is in israel")

[['NOUN'], ['NOUN'], ['VERB'], ['ADP'], ['NOUN']]

In [44]:
get_pos("i know how to write code")

[['PRON'], ['VERB'], ['ADV'], ['PRT'], ['VERB'], ['NOUN']]

#### Do the results look reasonable to you?

Well,  yes


## BONUS:
### Calculate the accuracy (proportion of POS tags predicted correctly) on the testing data.

Ctreating word_list and tags_list from test set

In [45]:
words_test = []
tags_test = []
for x in test:
    for w in x:
        if w[0].lower() in vocab:
            words_test.append(w[0].lower())
        else:
            words_test.append('[UNK]')
        tags_test.append(w[1])

Making prediction

In [46]:
pos_list = []

x = np.array([words_indx[k] for k in words_test]).reshape(-1, 1)
_, tag_decode = pos_model.decode(x)

for i in tag_decode:
    pos_list.append([u for u in tag_indx if tag_indx[u] == i])


y_pred = pos_list

Calculating accuracy

In [47]:
print('The Accuracy On The Testing Data.',round(accuracy_score(tags_test, y_pred),2))

The Accuracy On The Testing Data. 0.92
